Reference articles - 
- https://www.businessinsider.com/kanye-wests-turbulent-9-year-history-with-adidas-2022-11#2013-1

- Using pretrained model from huggingface - https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment

- https://aclanthology.org/2020.findings-emnlp.148/

- https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis

In [ ]:
# !pip install spacy
# !pip install nltk
# !pip install transformers
# !pip install torch

In [ ]:
import pandas as pd
import numpy as np
import string
string.punctuation
import spacy
import nltk
import seaborn as sns 
import warnings
import re
import datetime
warnings.filterwarnings('ignore')

In [ ]:
import nltk
# nltk.download()
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request
import torch
from transformers import pipeline



In [ ]:
# help("modules")

Data collected from US Weekly

In [ ]:
# TODO - change the filepath of the data local to your computer.

filepath = 'data_usweekly_final_2.csv'
df = pd.read_csv(filepath)
df.head()

,Unnamed: 0,Date,Link,Header,Article
0,0,"March 26, 2023",https://www.usmagazine.com/celebrity-news/pict...,"Jason Momoa, Olivia Munn and More Stars Who’ve...","['From fans to lovers! Jason Momoa, Olivia Mun..."
1,1,"March 17, 2023",https://www.usmagazine.com/celebrity-moms/pict...,"Kim Kardashian, Jimmy Fallon and More Celebrit...",['Pursuing surrogacy! Lance Bass and more\xa0c...
2,2,"March 13, 2023",https://www.usmagazine.com/entertainment/pictu...,Celebrity Exes Who Attended the Same Awards Sh...,['So awkward or super supportive? Stars often ...
3,3,"March 10, 2023",https://www.usmagazine.com/celebrity-news/news...,Kim Kardashian Is ‘Glad’ Ex Kanye West Has Fou...,['Wishing him the best. Kim Kardashian is supp...
4,4,"March 10, 2023",https://www.usmagazine.com/celebrity-news/pict...,Kanye West and Bianca Censori’s Relationship T...,['A whirlwind romance. Kanye West‘s love life ...


In [ ]:
# Removing Null Values

# df.Article.isna().sum() #5 Articles were null
df.dropna(inplace=True)
# df.Article.isna().sum() 

In [ ]:
# df.columns
df = df[['Date', 'Link', 'Header', 'Article']]

#### Data Cleaning

The data collected had some non english characters, some sentences that were advertising for the website and some unnecessary apostrophes. Those needed to be cleaned out and the collected sentences from the scraping had to be mergerd together to form a single article text.

In [ ]:
df['Article'] = df['Article'].apply(lambda x: x.strip('][').split(', '))

In [ ]:
article_str = []
for i in range(len(df)):
    try:
        article = df.Article[i]
        sub_list = df.Article[i][-10:]
        index=-10
        for item in sub_list:
            if '"Sign up for Us Weekly' in item or \
            'For access to all our exclusive celebrity videos and interviews' in item or\
            'Keep scrolling' in item or\
            'Scroll down' in item or\
            'In order to view the video' in item or\
            'Hear the Us Weekly editors break down' in item:
                break
            index += 1
        article = article[:index]
        article = [line[1:] if line[0] == "'" else line for line in article]
        article = [line[:-1] if line[-1] == "'" else line for line in article]
        article = (' ').join(article)
        article = article.replace(u'\\xa0', u' ')
        article_str.append(article)
    except:
        # to check number of failed attempts
        print('here')
        article_str.append(float("nan"))

here
here
here
here
here


In [ ]:
df['Article_str'] = article_str
df = df.dropna(subset=['Article_str'])

In [ ]:
df.head(2)

,Date,Link,Header,Article,Article_str
0,"March 26, 2023",https://www.usmagazine.com/celebrity-news/pict...,"Jason Momoa, Olivia Munn and More Stars Who’ve...","['From fans to lovers! Jason Momoa, Olivia Mun...",From fans to lovers! Jason Momoa Olivia Munn a...
1,"March 17, 2023",https://www.usmagazine.com/celebrity-moms/pict...,"Kim Kardashian, Jimmy Fallon and More Celebrit...",['Pursuing surrogacy! Lance Bass and more\xa0c...,Pursuing surrogacy! Lance Bass and more celebr...


### Perform Named Entity Recognition using spacy

To find out which articles are relevant to the person of interest - Kanye West, NER was performed to get a sense of all the people mentioned in the article. With furter investigation, we found out that it was worth performing further analysis on the articles that only mentioned Kanye. 

In [ ]:
# Needed for getting english vocabulary
# !python -m spacy download en_core_web_sm

In [ ]:
def get_person_ner(sentence):
    '''
    Given a text data, it perform Named Entity
    Recognition and collects all the PERSON entities 
    from the text
    
    Parameters: text data(str)
    Returns: list of all the PERSON entities found(list)
    '''
    try:
        nlp = spacy.load('en_core_web_sm')
        doc = nlp(sentence)
        person_set = set()
        for ent in doc.ents:
            if ent.label_ == 'PERSON':
                person_set.add(ent.text.lower())
    #             print(ent.text, ent.start_char, ent.end_char, ent.label_)
        return list(person_set)
    except: 
        return []

In [ ]:
# Getting PERSON names for headline

df['headline_person_ner'] = df.Header.apply(lambda x: get_person_ner(x))
df.head()

,Date,Link,Header,Article,Article_str,headline_person_ner
0,"March 26, 2023",https://www.usmagazine.com/celebrity-news/pict...,"Jason Momoa, Olivia Munn and More Stars Who’ve...","['From fans to lovers! Jason Momoa, Olivia Mun...",From fans to lovers! Jason Momoa Olivia Munn a...,"[jason momoa, olivia munn]"
1,"March 17, 2023",https://www.usmagazine.com/celebrity-moms/pict...,"Kim Kardashian, Jimmy Fallon and More Celebrit...",['Pursuing surrogacy! Lance Bass and more\xa0c...,Pursuing surrogacy! Lance Bass and more celebr...,"[kim kardashian, jimmy fallon]"
2,"March 13, 2023",https://www.usmagazine.com/entertainment/pictu...,Celebrity Exes Who Attended the Same Awards Sh...,['So awkward or super supportive? Stars often ...,So awkward or super supportive? Stars often at...,"[brad pitt, selena gomez, jennifer aniston, ju..."
3,"March 10, 2023",https://www.usmagazine.com/celebrity-news/news...,Kim Kardashian Is ‘Glad’ Ex Kanye West Has Fou...,['Wishing him the best. Kim Kardashian is supp...,Wishing him the best. Kim Kardashian is suppor...,[kim kardashian is ‘glad]
4,"March 10, 2023",https://www.usmagazine.com/celebrity-news/pict...,Kanye West and Bianca Censori’s Relationship T...,['A whirlwind romance. Kanye West‘s love life ...,A whirlwind romance. Kanye West‘s love life be...,[]


In [ ]:
# Getting PERSON names for article content

df['article_person_ner'] = df.Article_str.apply(lambda x: get_person_ner(x))
df.head()

,Date,Link,Header,Article,Article_str,headline_person_ner,article_person_ner
0,"March 26, 2023",https://www.usmagazine.com/celebrity-news/pict...,"Jason Momoa, Olivia Munn and More Stars Who’ve...","['From fans to lovers! Jason Momoa, Olivia Mun...",From fans to lovers! Jason Momoa Olivia Munn a...,"[jason momoa, olivia munn]","[aquaman, halsey, jason, anna marie tendler, s..."
1,"March 17, 2023",https://www.usmagazine.com/celebrity-moms/pict...,"Kim Kardashian, Jimmy Fallon and More Celebrit...",['Pursuing surrogacy! Lance Bass and more\xa0c...,Pursuing surrogacy! Lance Bass and more celebr...,"[kim kardashian, jimmy fallon]","[lance bass, turchin, michael turchin, bass]"
2,"March 13, 2023",https://www.usmagazine.com/entertainment/pictu...,Celebrity Exes Who Attended the Same Awards Sh...,['So awkward or super supportive? Stars often ...,So awkward or super supportive? Stars often at...,"[brad pitt, selena gomez, jennifer aniston, ju...","[brad pitt, pitt, lambert, miranda lambert, gw..."
3,"March 10, 2023",https://www.usmagazine.com/celebrity-news/news...,Kim Kardashian Is ‘Glad’ Ex Kanye West Has Fou...,['Wishing him the best. Kim Kardashian is supp...,Wishing him the best. Kim Kardashian is suppor...,[kim kardashian is ‘glad],"[kim kardashian, pete davidson, north, kim, ka..."
4,"March 10, 2023",https://www.usmagazine.com/celebrity-news/pict...,Kanye West and Bianca Censori’s Relationship T...,['A whirlwind romance. Kanye West‘s love life ...,A whirlwind romance. Kanye West‘s love life be...,[],"[kim kardashian, censori kardashian, kanye, we..."


In [ ]:
# Filter article and document based on Names of celebrities of interest. 

kanye_keywords = ['west', 'kanye', 'kanye west', 'ye']
kim_keywords = ['kim', 'kardashian','kim kardashian']
df['kim_in_header'] = [any(elem in x for elem in kim_keywords) for x in df['headline_person_ner']]
df['kim_in_article'] = [any(elem in x for elem in kim_keywords) for x in df['article_person_ner']]
df['kanye_in_header'] = [any(elem in x for elem in kanye_keywords) for x in df['headline_person_ner']]
df['kanye_in_article'] = [any(elem in x for elem in kanye_keywords) for x in df['article_person_ner']]
df.head()

,Date,Link,Header,Article,Article_str,headline_person_ner,article_person_ner,kim_in_header,kim_in_article,kanye_in_header,kanye_in_article
0,"March 26, 2023",https://www.usmagazine.com/celebrity-news/pict...,"Jason Momoa, Olivia Munn and More Stars Who’ve...","['From fans to lovers! Jason Momoa, Olivia Mun...",From fans to lovers! Jason Momoa Olivia Munn a...,"[jason momoa, olivia munn]","[aquaman, halsey, jason, anna marie tendler, s...",False,False,False,False
1,"March 17, 2023",https://www.usmagazine.com/celebrity-moms/pict...,"Kim Kardashian, Jimmy Fallon and More Celebrit...",['Pursuing surrogacy! Lance Bass and more\xa0c...,Pursuing surrogacy! Lance Bass and more celebr...,"[kim kardashian, jimmy fallon]","[lance bass, turchin, michael turchin, bass]",True,False,False,False
2,"March 13, 2023",https://www.usmagazine.com/entertainment/pictu...,Celebrity Exes Who Attended the Same Awards Sh...,['So awkward or super supportive? Stars often ...,So awkward or super supportive? Stars often at...,"[brad pitt, selena gomez, jennifer aniston, ju...","[brad pitt, pitt, lambert, miranda lambert, gw...",False,False,False,False
3,"March 10, 2023",https://www.usmagazine.com/celebrity-news/news...,Kim Kardashian Is ‘Glad’ Ex Kanye West Has Fou...,['Wishing him the best. Kim Kardashian is supp...,Wishing him the best. Kim Kardashian is suppor...,[kim kardashian is ‘glad],"[kim kardashian, pete davidson, north, kim, ka...",False,True,False,True
4,"March 10, 2023",https://www.usmagazine.com/celebrity-news/pict...,Kanye West and Bianca Censori’s Relationship T...,['A whirlwind romance. Kanye West‘s love life ...,A whirlwind romance. Kanye West‘s love life be...,[],"[kim kardashian, censori kardashian, kanye, we...",False,True,False,True


In [ ]:
print(df['kim_in_header'].value_counts())
print(df['kim_in_article'].value_counts())
print(df['kanye_in_header'].value_counts())
print(df['kanye_in_article'].value_counts())

False    1385
True      585
Name: kim_in_header, dtype: int64
True     1509
False     461
Name: kim_in_article, dtype: int64
False    1728
True      242
Name: kanye_in_header, dtype: int64
True     1338
False     632
Name: kanye_in_article, dtype: int64


In [ ]:
len(df)

1404

- Filtered articles associated only with Kanye West mentions either in the header or in the article content. 

In [ ]:
len(df[(df['kanye_in_article'] == True) & (df['kanye_in_header'] == True)])

176

In [ ]:
df = df[(df['kanye_in_article'] == True) | (df['kanye_in_header'] == True)]
df.reset_index(drop=True, inplace=True)

In [ ]:
len(df)

1404

Preprocess data -
- remove punctuations
- remove stop words
- stemming
- lemmatization


resource - https://www.analyticsvidhya.com/blog/2021/06/text-preprocessing-in-nlp-with-python-codes/


In [ ]:
def remove_punctuation(text):
    try:
        punctuationfree="".join([i for i in text if i not in string.punctuation])
        punctuationfree = punctuationfree.lower()
        return punctuationfree
    except:
        return text
#storing the puntuation free text
df['no_punc_data']= df['Article_str'].apply(lambda x:remove_punctuation(x))
# df.head()

In [ ]:
def tokenization(text):
    try:
        tokens = text.split()
        tokens = [(str(re.sub("[^a-zA-Z]+", "", word))) for word in tokens]
        tokens = [token for token in tokens if len(token)>0]
        return tokens
    except:
        return text
#applying function to the column
df['msg_tokenized']= df['no_punc_data'].apply(lambda x: tokenization(x))

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    try:
        output= [i for i in text if i not in stopwords]
#         print('here')
        return output
    except:
        return text
df['no_stopwords']= df['msg_tokenized'].apply(lambda x:remove_stopwords(x))


In [ ]:
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()
def stemming(text):
    try:
        stem_text = [porter_stemmer.stem(word) for word in text]
        return stem_text
    except:
        return text
df['msg_stemmed']=df['no_stopwords'].apply(lambda x: stemming(x))


In [ ]:
wordnet_lemmatizer = WordNetLemmatizer()
def lemmatizer(text):
    try:
        lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
        return lemm_text
    except:
        return text
df['msg_lemmatized']=df['msg_stemmed'].apply(lambda x:lemmatizer(x))


Using pretrained model - VADER - It is implementented in nltk as Sentiment Intensity Analyzer 

- VADER (Valence Aware Dictionary and Sentiment Reasoner) is a lexicon and rule-based sentiment analysis tool that is specifically designed to detect sentiments expressed in social media.

References
- https://towardsdatascience.com/the-most-favorable-pre-trained-sentiment-classifiers-in-python-9107c06442c6
- Paper - https://ojs.aaai.org/index.php/ICWSM/article/view/14550

In [ ]:
sia = SentimentIntensityAnalyzer()

In [ ]:
df = df.dropna(subset=['Article_str'])
df.reset_index(drop=True, inplace=True)

In [ ]:
df['nltk_score'] = df['Article_str'].apply(lambda x: sia.polarity_scores(x))
df.head()

,Date,Link,Header,Article,Article_str,headline_person_ner,article_person_ner,kim_in_header,kim_in_article,kanye_in_header,kanye_in_article,no_punc_data,msg_tokenized,no_stopwords,msg_stemmed,msg_lemmatized,nltk_score
0,"March 10, 2023",https://www.usmagazine.com/celebrity-news/news...,Kim Kardashian Is ‘Glad’ Ex Kanye West Has Fou...,['Wishing him the best. Kim Kardashian is supp...,Wishing him the best. Kim Kardashian is suppor...,[kim kardashian is ‘glad],"[kim kardashian, pete davidson, north, kim, ka...",False,True,False,True,wishing him the best kim kardashian is support...,"[wishing, him, the, best, kim, kardashian, is,...","[wishing, best, kim, kardashian, supportive, e...","[wish, best, kim, kardashian, support, exhusba...","[wish, best, kim, kardashian, support, exhusba...","{'neg': 0.007, 'neu': 0.846, 'pos': 0.147, 'co..."
1,"March 10, 2023",https://www.usmagazine.com/celebrity-news/pict...,Kanye West and Bianca Censori’s Relationship T...,['A whirlwind romance. Kanye West‘s love life ...,A whirlwind romance. Kanye West‘s love life be...,[],"[kim kardashian, censori kardashian, kanye, we...",False,True,False,True,a whirlwind romance kanye west‘s love life bec...,"[a, whirlwind, romance, kanye, wests, love, li...","[whirlwind, romance, kanye, wests, love, life,...","[whirlwind, romanc, kany, west, love, life, be...","[whirlwind, romanc, kany, west, love, life, be...","{'neg': 0.025, 'neu': 0.881, 'pos': 0.094, 'co..."
2,"February 17, 2023",https://www.usmagazine.com/celebrity-moms/pict...,Celeb Kids Meeting Their Siblings for the 1st ...,['Becoming big brothers and sisters. As celebr...,Becoming big brothers and sisters. As celebrit...,"[lucy cohen, ben]","[saint, kardashian, kim kardashian, psalm, luc...",False,True,False,True,becoming big brothers and sisters as celebrity...,"[becoming, big, brothers, and, sisters, as, ce...","[becoming, big, brothers, sisters, celebrity, ...","[becom, big, brother, sister, celebr, parent, ...","[becom, big, brother, sister, celebr, parent, ...","{'neg': 0.032, 'neu': 0.831, 'pos': 0.137, 'co..."
3,"February 14, 2023",https://www.usmagazine.com/celebrity-news/pict...,Celebrities’ Most Elaborate Valentine’s Day Gi...,['Valentine’s Day is a time to show loved ones...,Valentine’s Day is a time to show loved ones h...,[kim],"[kim kardashian, travis scott, kenny g, valent...",True,True,False,True,valentine’s day is a time to show loved ones h...,"[valentines, day, is, a, time, to, show, loved...","[valentines, day, time, show, loved, ones, muc...","[valentin, day, time, show, love, one, much, t...","[valentin, day, time, show, love, one, much, t...","{'neg': 0.01, 'neu': 0.776, 'pos': 0.214, 'com..."
4,"February 14, 2023",https://www.usmagazine.com/celebrity-moms/pict...,"North West’s Paintings, Drawings and More: Eve...",['Such an artist! North West has wowed her fam...,Such an artist! North West has wowed her famil...,"[kim kardashian, daughter has, kanye west’s]","[kris jenner, saint, kim kardashian, ice spice...",True,True,False,True,such an artist north west has wowed her family...,"[such, an, artist, north, west, has, wowed, he...","[artist, north, west, wowed, family, world, im...","[artist, north, west, wow, famili, world, impr...","[artist, north, west, wow, famili, world, impr...","{'neg': 0.015, 'neu': 0.819, 'pos': 0.165, 'co..."


- pos, neu, and neg scores are ratios for proportions of text that fall in each category (so these should all add up to be 1 or close to it with float operation).
- The compound score is computed by summing the valence scores of each word in the lexicon, adjusted according to the rules, and then normalized to be between -1 (most extreme negative) and 1 (most extreme positive). This is the most useful metric if you want a single measure of sentiment for a given sentence.

In [ ]:
df['nltk_positive'] = df['nltk_score'].apply(lambda x: x['pos'])
df['nltk_negative'] = df['nltk_score'].apply(lambda x: x['neg'])
df['nltk_neutral'] = df['nltk_score'].apply(lambda x: x['neu'])
df['nltk_compound'] = df['nltk_score'].apply(lambda x: x['compound'])


Using pretrained model from huggingface - A Transformer-based library for SocialNLP tasks.
- Model from Huggingface - https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis
- Paper - https://arxiv.org/abs/2106.09462


In [ ]:
model = 'finiteautomata/bertweet-base-sentiment-analysis'

In [ ]:
nlp = pipeline("sentiment-analysis", model="finiteautomata/bertweet-base-sentiment-analysis", tokenizer="finiteautomata/bertweet-base-sentiment-analysis")

# Example social media article to analyze
article = "I love this product so much! It's the best thing I've ever bought."

# Perform sentiment analysis on the article
result = nlp(article)

# Print the sentiment label and score
print(f"Sentiment: {result[0]['label']}, Score: {result[0]['score']}")

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


Sentiment: POS, Score: 0.9923118948936462


In [ ]:
def get_huggingface_sent_score(text):
    '''
    Given an article text data, it breaks the large text data
    into smaller chunks. A huggingface pretrained model is used 
    that performs Sentiment analysis and scoring on each of the 
    chunks
    
    Parameters: text data(str)
    
    Returns: a dictionary of the following structure
    {'positive': {'count': 16, 'score_average': 0.92, 'score_median': 0.96,'score_max': 0.98},
    'negative': {'count': 5, 'score_average': 0.91, 'score_median': 0.92,'score_max': 0.99},
    'neutral': {'count': 8, 'score_average': 0.90, 'score_median': 0.88,'score_max': 0.91} }
     
     where count represent number of chunks of the text that are positive/negative/neutral,
     score_average represent the average sentiment score of the chuncks of each kind,
     score_median represent the median sentiment score of the chuncks of each kind,
     score_max represent the maximum sentiment score of the chuncks of each kind
    
    '''
    try:
        sample = text.split('.')
        pos = []
        neg = []
        neu= []
        pos_count = 0
        neg_count = 0
        neu_count = 0
        for line in sample:
            result = nlp(line)
            if result[0]['label'] == 'POS':
                pos.append(result[0]['score'])
                pos_count += 1
            if result[0]['label'] == 'NEG':
                neg.append(result[0]['score'])
                neg_count += 1
            if result[0]['label'] == 'NEU':
                neu.append(result[0]['score'])
                neu_count += 1
        #     print(f"Sentiment: {result[0]['label']}, Score: {result[0]['score']}")

        positive = {'count': pos_count, 'score_average': np.round(np.mean(pos),2), 'score_median': np.round(np.median(pos),2),'score_max': np.round(np.max(pos),2)}
        negative = {'count': neg_count, 'score_average': np.round(np.mean(neg),2), 'score_median': np.round(np.median(neg),2), 'score_max': np.round(np.max(neg),2)}
        neutral = {'count': neu_count, 'score_average': np.round(np.mean(neu),2), 'score_median': np.round(np.median(neu),2), 'score_max': np.round(np.max(neu),2)}
        return {'positive':positive, 'negative': negative, 'neutral':neutral }
    except:
        return float('nan')
#     print(f"Sentiment: POS, positive_count: {pos_count}, score_average: {np.round(np.mean(pos),2)}, score_median: {np.round(np.median(pos),2)},score_max: {np.round(np.max(pos),2)}")
#     print(f"Sentiment: NEG, negative_count: {neg_count}, score_average: {np.round(np.mean(neg),2)}, score_median: {np.round(np.median(neg),2)}, score_max: {np.round(np.max(neg),2)}")
#     print(f"Sentiment: NEU, neutral_count: {neu_count}, score_average: {np.round(np.mean(neu),2)}, score_median: {np.round(np.median(neu),2)}, score_max: {np.round(np.max(neu),2)}")

In [ ]:
df['hug_face_bertweet'] = df['Article_str'].apply(lambda x: get_huggingface_sent_score(x))

Token indices sequence length is longer than the specified maximum sequence length for this model (141 > 128). Running this sequence through the model will result in indexing errors


In [ ]:
print(len(df))
print(len(df[df.hug_face_bertweet.isnull()]))

1404
446


In [ ]:
df = df.dropna(subset=['hug_face_bertweet'])
# df_new.drop(['level_0', 'index'], axis=1, inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
len(df)

958

In [ ]:
df['hg_positive'] = df['hug_face_bertweet'].apply(lambda x: x['positive'])
df['hg_negative'] = df['hug_face_bertweet'].apply(lambda x: x['negative'])
df['hg_neutral'] = df['hug_face_bertweet'].apply(lambda x: x['neutral'])


In [ ]:
df['hg_positive_count'] = df['hg_positive'].apply(lambda x: x['count'])
df['hg_negative_count'] = df['hg_negative'].apply(lambda x: x['count'])
df['hg_neutral_count'] = df['hg_neutral'].apply(lambda x: x['count'])


In [ ]:
df['hg_positive_score_avg'] = df['hg_positive'].apply(lambda x: x['score_average'])
df['hg_negative_score_avg'] = df['hg_negative'].apply(lambda x: x['score_average'])
df['hg_neutral_score_avg'] = df['hg_neutral'].apply(lambda x: x['score_average'])


In [ ]:
df['hg_positive_score_med'] = df['hg_positive'].apply(lambda x: x['score_median'])
df['hg_negative_score_med'] = df['hg_negative'].apply(lambda x: x['score_median'])
df['hg_neutral_score_med'] = df['hg_neutral'].apply(lambda x: x['score_median'])


In [ ]:
df['hg_positive_score_max'] = df['hg_positive'].apply(lambda x: x['score_max'])
df['hg_negative_score_max'] = df['hg_negative'].apply(lambda x: x['score_max'])
df['hg_neutral_score_max'] = df['hg_neutral'].apply(lambda x: x['score_max'])

In [ ]:
df['year'] = df.Date.apply(lambda x: int(x.split()[2]))
df['day'] = df.Date.apply(lambda x: int(x.split()[1][:-1]))
df['month'] = df.Date.apply(lambda x: datetime.datetime.strptime(x.split()[0], "%B").month)
df['date'] = pd.to_datetime(df[['day', 'month', 'year']])
df['week'] = df['date'].apply(lambda x: x.strftime("%V"))
df.head()

,Date,Link,Header,Article,Article_str,headline_person_ner,article_person_ner,kim_in_header,kim_in_article,kanye_in_header,...,hg_negative_score_med,hg_neutral_score_med,hg_positive_score_max,hg_negative_score_max,hg_neutral_score_max,year,day,month,date,week
0,"March 10, 2023",https://www.usmagazine.com/celebrity-news/news...,Kim Kardashian Is ‘Glad’ Ex Kanye West Has Fou...,['Wishing him the best. Kim Kardashian is supp...,Wishing him the best. Kim Kardashian is suppor...,[kim kardashian is ‘glad],"[kim kardashian, pete davidson, north, kim, ka...",False,True,False,...,0.97,0.94,0.98,0.97,0.97,2023,10,3,2023-03-10,10
1,"March 10, 2023",https://www.usmagazine.com/celebrity-news/pict...,Kanye West and Bianca Censori’s Relationship T...,['A whirlwind romance. Kanye West‘s love life ...,A whirlwind romance. Kanye West‘s love life be...,[],"[kim kardashian, censori kardashian, kanye, we...",False,True,False,...,0.97,0.92,0.99,0.97,0.97,2023,10,3,2023-03-10,10
2,"February 17, 2023",https://www.usmagazine.com/celebrity-moms/pict...,Celeb Kids Meeting Their Siblings for the 1st ...,['Becoming big brothers and sisters. As celebr...,Becoming big brothers and sisters. As celebrit...,"[lucy cohen, ben]","[saint, kardashian, kim kardashian, psalm, luc...",False,True,False,...,0.91,0.84,0.99,0.97,0.97,2023,17,2,2023-02-17,07
3,"February 14, 2023",https://www.usmagazine.com/celebrity-moms/pict...,"North West’s Paintings, Drawings and More: Eve...",['Such an artist! North West has wowed her fam...,Such an artist! North West has wowed her famil...,"[kim kardashian, daughter has, kanye west’s]","[kris jenner, saint, kim kardashian, ice spice...",True,True,False,...,0.89,0.92,0.99,0.89,0.97,2023,14,2,2023-02-14,07
4,"February 5, 2023",https://www.usmagazine.com/entertainment/pictu...,Celebrity PDA at the Grammy Awards Through the...,['Hot and heavy! These famous couples weren’t ...,Hot and heavy! These famous couples weren’t af...,"[jennifer lopez, kylie jenner]","[alicia keys, julia michaels, travis scott, ca...",False,True,False,...,0.76,0.88,0.99,0.98,0.96,2023,5,2,2023-02-05,05


In [ ]:
df.to_csv('Sentiment_Analysis_Data.csv')

In [ ]:
final_sentiment = []
final_score = []
final_sentiment_pos_neg = []
final_score_pos_neg = []
for i in range(len(df)):
    d = df.hug_face_bertweet[i]
    pos = d['positive']
    pos['key'] = 'positive'
    neg = d['negative']
    neg['key'] = 'negative'
    neu = d['neutral']
    neu['key'] = 'neutral'
    l = [pos,neg,neu]
    pos_neg = [pos,neg]
    new_l_pos_neg = sorted(pos_neg, key=lambda x: x['count']) 
    new_l = sorted(l, key=lambda x: x['count']) 
    final_sentiment.append(new_l[2]['key'])
    final_score.append(new_l[2]['score_average'])
    final_sentiment_pos_neg.append(new_l_pos_neg[1]['key'])
    final_score_pos_neg.append(new_l_pos_neg[1]['score_average'])
    

In [ ]:
df['final_sentiment'] = final_sentiment
df['final_score'] = final_score
df['final_sentiment_pos_neg'] = final_sentiment_pos_neg
df['final_score_pos_neg'] = final_score_pos_neg

In [ ]:
df.head()

,Date,Link,Header,Article,Article_str,headline_person_ner,article_person_ner,kim_in_header,kim_in_article,kanye_in_header,...,hg_neutral_score_max,year,day,month,date,week,final_sentiment,final_score,final_sentiment_pos_neg,final_score_pos_neg
0,"March 10, 2023",https://www.usmagazine.com/celebrity-news/news...,Kim Kardashian Is ‘Glad’ Ex Kanye West Has Fou...,['Wishing him the best. Kim Kardashian is supp...,Wishing him the best. Kim Kardashian is suppor...,[kim kardashian is ‘glad],"[kim kardashian, pete davidson, north, kim, ka...",False,True,False,...,0.97,2023,10,3,2023-03-10,10,neutral,0.88,positive,0.83
1,"March 10, 2023",https://www.usmagazine.com/celebrity-news/pict...,Kanye West and Bianca Censori’s Relationship T...,['A whirlwind romance. Kanye West‘s love life ...,A whirlwind romance. Kanye West‘s love life be...,[],"[kim kardashian, censori kardashian, kanye, we...",False,True,False,...,0.97,2023,10,3,2023-03-10,10,neutral,0.88,positive,0.97
2,"February 17, 2023",https://www.usmagazine.com/celebrity-moms/pict...,Celeb Kids Meeting Their Siblings for the 1st ...,['Becoming big brothers and sisters. As celebr...,Becoming big brothers and sisters. As celebrit...,"[lucy cohen, ben]","[saint, kardashian, kim kardashian, psalm, luc...",False,True,False,...,0.97,2023,17,2,2023-02-17,07,neutral,0.83,positive,0.91
3,"February 14, 2023",https://www.usmagazine.com/celebrity-moms/pict...,"North West’s Paintings, Drawings and More: Eve...",['Such an artist! North West has wowed her fam...,Such an artist! North West has wowed her famil...,"[kim kardashian, daughter has, kanye west’s]","[kris jenner, saint, kim kardashian, ice spice...",True,True,False,...,0.97,2023,14,2,2023-02-14,07,neutral,0.86,positive,0.90
4,"February 5, 2023",https://www.usmagazine.com/entertainment/pictu...,Celebrity PDA at the Grammy Awards Through the...,['Hot and heavy! These famous couples weren’t ...,Hot and heavy! These famous couples weren’t af...,"[jennifer lopez, kylie jenner]","[alicia keys, julia michaels, travis scott, ca...",False,True,False,...,0.96,2023,5,2,2023-02-05,05,neutral,0.80,positive,0.80


In [ ]:
df.to_csv('Sentiment_Analysis_Data.csv')